# SPECFEM Users Workshop -- Day 3 (Oct. 7, 2022)

## Part 3: Seismic Imaging

TO DO:
- Run a manual model update by guessing the perturbation as the negative gradient? Try to follow golden rule or smoething
- Attempt one or two updates manually, checking waveform misfit each time
- Then run Example 1 to show HH models
- Run Example 2 for inversion
- Allow users to play around with starting model, preprocessing etc.

- In this notebook we will bring all of the previous material together run a full seismic inversion to update a starting model. 
!!! ADD TEXT HERE !!!

-----------

### Relevant Information

- These instructions should be run from inside the Docker container, using Jupyter Lab. The Docker container should have the adjTomo toolkit installed (SeisFlows, Pyatoa, PySEP), as well as SPECFEM2D and SPECEFM3D compiled with MPI. 

**Relevant Links:** 
- Day 3 Slides !!! ADD THIS !!!
- [Today's Notebook](https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/day_3a_imaging.ipynb)
- Completed Notebook !!! ADD THIS !!!

**Jupyter Quick Tips:**

- **Run cells** one-by-one by hitting the $\blacktriangleright$ button at the top, or by hitting `Shift + Enter`
- **Run all cells** by hitting the $\blacktriangleright\blacktriangleright$ button at the top, or by running `Run -> Run All Cells`
- **Currently running cells** that are still processing will have a `[*]` symbol next to them
- **Finished cells** will have a `[1]` symbol next to them. The number inside the brackets represents what order this cell has been run in.
- Commands that start with `!` are Bash commands (i.e., commands you would run from the terminal)
- Commands that start with `%` are Jupyter Magic commands.


## 1) Background

!!! TO DO !!!
Potential topics:
- Nonlinear optimization algorithms
- Line searches
- Model perturbations

## 2) Setting Up 

- We want to set up a clean working directory to run SPECFEM2D inside. This will help us preserve our cloned repository and reduce file clutter.
- We will use SeisFlows to automate an inversion workflow **up to** kernel generation. 
- SeisFlows will run a forward simulation, calculate misfit, and run an adjoint simulation.
- Experimental setup is one source, one receiver, comparing the homogeneous halfspace and checkerboard models.
- **Objective**:Generate a misfit kernel which we can use to update our model manually. 

>__NOTE:__ We will be doing all our work in the directory /home/scoped/work_day_3. All the following cells assume that we are in this directory, so you must evaluate the '%cd' command to ensure that cells work as expected.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
from scipy.integrate import simps
from seisflows.tools.specfem import Model

In [ ]:
# make sure we're in an empty working directory
! mkdir -p /home/scoped/work/day_3/manual_update
%cd /home/scoped/work/day_3/manual_update

# Run the example and stop after adjoint simulation
! seisflows examples setup 2 -r /home/scoped/specfem2d --event_id 1 --nsta 1 --niter 1 --with_mpi
! seisflows par stop_after evaluate_gradient_from_kernels
! seisflows submit

-----------

You will know that this workflow has completed successfully after the final log message   

```bash
2022-09-16 19:57:07 (I) | stop workflow at `stop_after`: evaluate_gradient_from_kernels
```

We can use SeisFlows to plot the Gradient and make sure it looks familiar. Under the hood, the `plot2d` command functions similarly to what we were doing in the previous notebook, that is: loading the underlying gradient using NumPy, and plotting it using Matplotlib.

In [ ]:
! seisflows plot2d GRADIENT_01 vs_kernel --savefig g_01_vs.png
Image("g_01_vs.png")

This kernel image is very similar to what we recovered after smoothing our misfit kernel in Day 2, and provides us a great jumping off point for looking into model updates.

## 3) Manually Updating a Velocity Model

- We will use this gradient that was generated using SeisFlows to update our initial model. 
- With only have one source receiver pair, we can immediately check if the misfit of the waveforms has decreased by solving for $\chi$.
- In SeisFlows, we treat models and gradients as linear arrays, so model update can be memory-intensive, but is not CPU intensive.

The Model update equation can be simple as 

$m_{i+1} = m_{i} - \alpha \times g$ 

Where $m_{i}$ is the current model, $m_{i+1}$ is the updated model, $g$ is the gradient and $\alpha$ is a scale factor.

### Initial Model ($m_i$)

The initial model is our homogeneous halfspace model

In [ ]:
# Load in the initial model binary files, plot
m_init = Model("output/MODEL_INIT")
print(f"INITIAL MODEL\n{m_init.model}")
m_init.plot2d("vs")

### Gradient ($g$)

The gradient is defined by the interaction of our forward and adjoint wavefields. It represents the gradient of the misfit function and provides a search direction for model update.

>__NOTE:__ Because the Gradient output directory does not contain coordinate information, we need to grab it from the model, which natively exports to disk with coordinate information.
Otherwise, loading in the Gradient using SeisFlows is identical to loading in models.

In [ ]:
# Load in Gradient binary files
gradient = Model("output/GRADIENT_01")

# Assign coordinate information which is used for plotting
gradient.coordinates = {}
gradient.coordinates["x"] = m_init.coordinates["x"]
gradient.coordinates["z"] = m_init.coordinates["z"]

print(f"GRADIENT\n{gradient.model}")
gradient.plot2d("vs_kernel")

### Updated Model ($m_{i+1}$)

The updated model can be found by scaling the gradient and adding the negative gradient to the initial model. 
Remember that the gradient represents maximum change, and since we are solving a minimization problem, we want to use the inverse gradient.

In [ ]:
# We can use the SeisFlows Model class to update the model directly
m_update = m_init.copy()

# No scaling is applied here
m_update.update(vector=m_init.vector - gradient.vector)

print(m_update.model)

m_update.plot2d("vs")

- We can see that because the gradient is **not** well scaled, the model updates only make slight changes to the Vs model ($\pm.0001$m/s give or take). 
- In order to make appreciable changes to the starting model, we need the **scale** the gradient. 
- There are many algorithms which provide scaling estimates for the gradient. 
- One thing we can try, is scaling by GTG$^{-1}$, or the inverse of the dot product of the gradient itself.

In [ ]:
# Caclulating GTG^-1
gtg = np.dot(gradient.vector, gradient.vector)
gtg_inverse = gtg ** -1 

print(f"GTG^-1 = {gtg_inverse:.2f}")

In [ ]:
# Update with a scaled gradient
m_update = m_init.copy()
m_update.update(vector=m_init.vector - (gtg_inverse * gradient.vector))

print(m_update.model)

m_update.plot2d("vs")

- The gradient is now more well scaled, and has updated our velocity model at most 25m/s. 
- We can now try to use this updated velocity model to generate **new** synthetics, and calculate waveform misfit. 
- If misfit has reduced from the original model, then we're getting somewhere!

## 4) Forward Simulations w/ Updated Model

- We now have an updated model stored in the Python parameter `m_update`. 
- We'll use this updated model to generate a new set of synthetics
- As in Day 2, misfit will be calculated using a waveform misfit. 
- **Objective**: Determine if we have reduced waveform misfit through model update

### Setup SPECFEM2D working directory
Let's start out by setting up a new SPECFEM2D working directory where we can manually run the SPECFEM binaries. This is very similar to Day 2.

In [ ]:
# Setup block for our SPECFEM2D working directory
! mkdir /home/scoped/work/day_3/specfem2d_workdir
%cd /home/scoped/work/day_3/specfem2d_workdir

# Symlink the binary files located in bin/
! ln -s /home/scoped/specfem2d/bin .
# Copy over the DATA/ directory with Par_file, SOURCE and STATIONS
! cp -r /home/scoped/specfem2d/EXAMPLES/Tape2007/DATA .
# Pick a pre-defined Par_file 
! cp DATA/Par_file_Tape2007_onerec DATA/Par_file
# Ensure we are using SOURCE #1
! cp DATA/SOURCE_001 DATA/SOURCE
# Ensure we are using only STATION #1
! head -1 DATA/STATIONS_checker > DATA/STATIONS
# Create the output directory
! mkdir OUTPUT_FILES

! ls

### Run SPECFEM2D to get model $m_i$ synthetics

- First we'll run SPECFEM2D to get model $m_i$ synthetics
- We'll later compare these to the updated model ($m_{i+1}$) synthetics
- We'll need to adjust some parameter file parameters before running

In [ ]:
# Tell SPECFEM to output binary model files
! seisflows sempar -P DATA/Par_file save_model binary
! seisflows sempar -P DATA/Par_file setup_with_binary_database 1
! seisflows sempar -P DATA/Par_file use_existing_stations .true.

# Run SPECFEM with the homogeneous halfspace model, 1 source, 1 station
! mpirun -n 1 bin/xmeshfem2D > OUTPUT_FILES/output_mesher.txt
! mpirun -n 1 bin/xspecfem2D > OUTPUT_FILES/output_solver.txt

In [ ]:
# A succesfully completed solver will write data files
! tail -n 30 OUTPUT_FILES/output_solver.txt

### Set Updated Model

- SPECEFM2D needs to be able to find the updated model we just created. Luckily we can use SeisFlows to write these files in SPECFEM-readable formats.
- SPECFEM2D expects model files to be in the DATA/ directory. 
- We can OVERWRITE the current model parameters 
- The `Par_file` parameter 'MODEL' must be set to 'gll', which tells SPECFEM to read existing model files

>__NOTE__: SPECEFEM**3D** expects model files to be the `LOCAL_PATH` directory, which usually points to OUTPUT_FILES/DATABASES_MPI

In [ ]:
# First we make sure that the original model files are stored somewhere
! mkdir MODEL_INIT
! cp -r DATA/*bin MODEL_INIT
! cp -r OUTPUT_FILES/*semd MODEL_INIT
! ls MODEL_INIT

In [ ]:
# Now we use SeisFlows to write out the model
print(m_update.parameters)

# This will overwrite the 'Vp' and 'Vs' binary files
m_update.write(path="DATA")

! ls -l DATA/*.bin

In [ ]:
# Update the `Par_file` parameter 'MODEL' to tell SPECFEM to read model files from DATA/
! seisflows sempar -P DATA/Par_file model gll

### Run SPECFEM2D with updated model

- We can now run `xmeshfem2D` and `xspecfem2D` to retrieve synthetics for our updated model
- Since we set `MODEL`==`gll`, SPECFEM2D will know to read the **updated** model files we just wrote
- This run will **overwrite** files stored in `OUTPUT_FILES/`. If you want to preserve these, you can copy the directory to a new path

In [ ]:
! mpirun -n 1 bin/xmeshfem2D > OUTPUT_FILES/output_mesher_update.txt
! mpirun -n 1 bin/xspecfem2D > OUTPUT_FILES/output_solver_update.txt

In [ ]:
# We can confirm that the updated model was used during the simulation by checking the solver log
! head -236 OUTPUT_FILES/output_solver_update.txt | tail -n 15

In [ ]:
# Make sure that the final model files are stored somewhere
! mkdir MODEL_UPDATE
! cp -r DATA/*.bin MODEL_UPDATE
! cp -r OUTPUT_FILES/*semd MODEL_UPDATE
! ls MODEL_UPDATE

## 5) Comparing Synthetics for Initial and Updated Models

- We now have 2 synthetic seismograms. 1 for the initial model, 1 for the updated model
- If we plot the two waveforms we should see that they differ
- The final test would be to check both synthetics against the True model synthetics (checkerboard model)
- **Objective**: Determine if our updated model has reduced waveform misfit

### Compare Initial and Update synthetics

We'll re-use some code snippets from Day 2 to plot synthetics

In [ ]:
t_init, d_init = np.loadtxt("MODEL_INIT/AA.S000000.BXY.semd").T
t_update, d_update = np.loadtxt("MODEL_UPDATE/AA.S000000.BXY.semd").T

# Plot both waveforms using Matplotlib
plt.plot(t_init, d_init, c="r", label="MODEL INIT; SYNTHETIC")
plt.plot(t_update, d_update, c="magenta", label="MODEL UPDATE; 'SYNTHETIC'")
plt.plot(t_init, d_init - d_update, c="b", label="SYNTHETIC DIFFERENCE")
plt.xlabel("Time [s]")
plt.ylabel("Displacement [m]")
plt.legend()
plt.show()

We can see tha the waveform difference is small but present.

### Generating True/Target model synthetics

- The final check here is to calculate waveform misfit for the initial and updated models, with respect to the Target checkerboard model.
- We will first need to generate checkerboard synthetics, which was done in Day 2
- We will also need to define our waveform misfit function

In [ ]:
# This new par file OVERWRITES the changes we made previously 
! cp -f DATA/Par_file_Tape2007_132rec_checker DATA/Par_file

! seisflows sempar -P DATA/Par_file NSTEP 5000  # to match the other Par_file
! seisflows sempar -P DATA/Par_file save_model binary
! seisflows sempar -P DATA/Par_file setup_with_binary_database 1

# Ensure that SPECFEM can find the checkerboard model by naming it correctly
! cp -f DATA/model_velocity.dat_checker DATA/proc000000_model_velocity.dat_input

! mpirun -n 1 bin/xmeshfem2D > OUTPUT_FILES/output_mesher_true.txt
! mpirun -n 1 bin/xspecfem2D > OUTPUT_FILES/output_solver_true.txt

In [ ]:
# Store True model binary files and synthetics
! mkdir MODEL_TRUE
! cp -r DATA/*.bin MODEL_TRUE
! cp -r OUTPUT_FILES/*semd MODEL_TRUE
! ls MODEL_TRUE

In [ ]:
# Lets plot waveforms again, but with the True model, which will be colored magenta
t_true, d_true = np.loadtxt("MODEL_TRUE/AA.S000000.BXY.semd").T

plt.plot(t_init, d_init, c="r", label="MODEL INIT; SYNTHETIC")
plt.plot(t_true, d_true, c="k", label="MODEL TRUE; 'DATA'")
plt.plot(t_update, d_update, c="magenta", label="MODEL UPDATE; 'SYNTHETIC'")
plt.plot(t_init, d_init - d_update, c="b", label="SYNTHETIC DIFFERENCE")
plt.xlabel("Time [s]")
plt.ylabel("Displacement [m]")
plt.legend()
plt.show()

### Calculating Misfit

- As in Day 2 we will calculate a waveform misfit, whose equation is: $ \chi = \frac{1}{2} \int [d(t)-s(t)]^2 dt~, $
- **Objective**: Determine if the model update has reduced waveform misfit




In [ ]:
# Integrate using scipy
dt = t_true[1] - t_true[0]  # represents the time step, or `dt`. Same for all waveforms

misfit_init = 1/2 * simps((d_true - d_init)**2, dx=dt)
misfit_update = 1/2 * simps((d_true - d_update)**2, dx=dt)

print(f"misfit_init=={misfit_init:.3E}")
print(f"misfit_true=={misfit_update:.3E}")
print(f"difference=={misfit_init - misfit_update:.3E}")


---------------

**Mission accomplished!**
- We have successfully reduced the waveform misfit for a single source receiver pair by updating our velocity model.
- The misfit reduction is small, and tied to the gradient and the scale factor chosen
- Users could increase the scale factor to achieve a more favorable misfit reduction
- Algorithmically, this step is encapsulated in the `line search`, which attempts to find an appropriate scale factor 
- There are many types of nonlinear optimization algorithms (e.g., gradient descent, L-BFGS) which handle this task
- **NOTE** that each evaluation of a line search requires re-running the forward simulation with a trial model
- Seismic tomography must be performed **iteratively**, i.e., we need to update our model repeatedly to find an optimal solution

## 6) Automating Inversions using SeisFlows

- Of course, now that we have gone through the rigorous manual exercise of updating a velocity model, we can see how SeisFlows automates this procedure.
- SeisFlows contains a built-in optimization library, which features gradient-descent, non-linear conjugate gradient (NLCG), and L-BFGS nonlinear optimization alogirthms
- This library takes care of the scaling of the gradient, automating the line search and model update
- Under the hood, SeisFlows essentialy performs what we just did, scaling the velocity model, re-running forward simulations and comparing misfit values
- Misfit is either calculated with the default preprocessing module, or using Pyatoa, a software designed specifically for waveform misfit quantification.

In [ ]:
! mkdir /home/scoped/work/day_3/sfexample_2
%cd /home/scoped/work/day_3/sfexample_2
! seisflows examples setup 2 -r /home/scoped/specfem2d --event_id 1 --niter 1 --nsta 1 --with_mpi

!!! analyze log message above here !!!

In [ ]:
! seisflows submit

-------------

The task will be finished when you see the log message:

```bash
////////////////////////////////////////////////////////////////////////////////
                             COMPLETE ITERATION 01                              
////////////////////////////////////////////////////////////////////////////////
2022-09-16 21:25:38 (I) | setting current iteration to: 2
```

In [ ]:
! echo "> output/ contains models, gradients, etc."
! ls output
! echo

! echo "> MODEL directories contain SPECFEM formatted model files"
! ls output/MODEL_INIT
! echo

! echo "> GRADIENT directories contain regularized misfit kernels"
! ls output/GRADIENT_01
! echo

In [ ]:
# Models/gradients can be plotted directly from the command line
! seisflows plot2d MODEL_01 vs --savefig m_01_vs.png
Image("m_01_vs.png")

Under the hood, SeisFlows is simply running a number of individual SPECFEM working directories

In [ ]:
! ls scratch/solver/001